# Функции

Содержание лекции:
1. Как объявлять функции
2. Утиная типизация в Julia
3. Мутирующие и немутирующие функции
4. Некоторые функции высокого порядка

## Как объявлять функции
Julia предоставляет несколько различных способов для определения функций. Первый способ - с использованием ключевых слов `function` и `end`.

In [1]:
function sayhi(name)
    println("Привет $name, рад тебя видеть!")
end

sayhi (generic function with 1 method)

In [2]:
function f(x)
    return x^2
end

f (generic function with 1 method)

В дальнейшем эти функции могут быть вызваны следующим образом:

In [3]:
sayhi("C-3PO")

Привет C-3PO, рад тебя видеть!


In [4]:
f(42)

1764

Альтернативный способ заключается в определении функии в одну строку

In [5]:
sayhi2(name) = println("Привет $(name)! Как твои дела?")

sayhi2 (generic function with 1 method)

In [6]:
f2(x) = x^3

f2 (generic function with 1 method)

In [7]:
sayhi2("R2D2")

Привет R2D2! Как твои дела?


In [8]:
f2(42)

74088

Последний способ заключается в использовании "анонимных" функций

In [9]:
sayhi3 = name -> println("Всем привет! И тебе, $(name), тоже!")

#11 (generic function with 1 method)

In [10]:
f3 = x -> x^4

#13 (generic function with 1 method)

In [11]:
sayhi3("Чуи")

Всем привет! И тебе, Чуи, тоже!


In [12]:
f3(42)

3111696

## Утиная типизация в Julia
*"Если это крякает как утка, то это утка."* <br><br>
Функции Julia будут работать с любыми входными данными, с которыми их определение имеет смысл. <br><br>
Например, `sayhi` работает точно также, если в качестве параметра `name` передать число

In [13]:
sayhi(55595472)

Привет 55595472, рад тебя видеть!


А функция `f` будет работать для квадратных матриц. 

In [14]:
A = rand(3, 3)
A

3×3 Matrix{Float64}:
 0.747696  0.151816  0.232916
 0.11829   0.296295  0.733418
 0.576365  0.674843  0.208035

In [15]:
f(A)

3×3 Matrix{Float64}:
 0.711252  0.315676  0.333949
 0.546209  0.600691  0.397436
 0.630676  0.427845  0.672465

`f` так же будет работать, если передать строку "hi", поскольку для строковых переменных определена операция `^`, возвращающая копирование строки

In [16]:
f("hi")

"hihi"

С другой стороны, `f` не будет работать с векторами. В отличии от операции `A^2`, которая имеет смысл, операция возведения в степень вектора `v^2` не имеет смысла с математической точки зрения.

In [17]:
v = rand(3)

3-element Vector{Float64}:
 0.837567537792885
 0.9382429419924768
 0.7952430890068428

In [18]:
# This won't work
f(v)

MethodError: MethodError: no method matching ^(::Vector{Float64}, ::Int64)

Closest candidates are:
  ^(!Matched::Float16, ::Integer)
   @ Base math.jl:1283
  ^(!Matched::Irrational{:ℯ}, ::Integer)
   @ Base mathconstants.jl:119
  ^(!Matched::BigInt, ::Integer)
   @ Base gmp.jl:654
  ...


## Мутирующие и немутирующие функции

Согласно договоренности, функции, название которых оканчивается на `!` изменяют параметры, которые в них передаются, а функции без `!` - не меняют.

Например, давайте посмотрим на различие между `sort` и `sort!`.


In [19]:
v = [3, 5, 2]

3-element Vector{Int64}:
 3
 5
 2

In [20]:
sort(v)

3-element Vector{Int64}:
 2
 3
 5

In [21]:
v

3-element Vector{Int64}:
 3
 5
 2

`sort(v)` возвращает отсортированный вектор `v`, однако `v` остается неизменным. <br><br>

С другой сторооны, когда мы вызываем `sort!(v)`, содержимое `v` сортируется внутри переменной.

In [22]:
sort!(v)

3-element Vector{Int64}:
 2
 3
 5

In [23]:
v

3-element Vector{Int64}:
 2
 3
 5

## Некоторые высокоуровневые функции

### map

`map` - это "высокоуровневая" функция в Julia, которая *принимает функцию* в качестве одного из аргументов. 
`map` применяет данную функцию к каждому элементу структуры, которую вы также передаете в функцию. Например, выполнив

```julia
map(f, [1, 2, 3])
```
вы получите массив, где функция `f` была применена последовательно к `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [24]:
map(f, [1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Здесь мы получили квадраты элементов массива `[1, 2, 3]`, а не "квадрат" массива `[1, 2, 3]`.

В случае, если у нас нет в наличии необходимой функции что бы передать ее в `map`, мы можем использовать анонимную функцию, которая создается "налету"

In [25]:
map(x -> x^5, [1, 2, 3])

3-element Vector{Int64}:
   1
  32
 243

И теперь у нас есть пятые степени массива `[1, 2, 3]`

### broadcast

`broadcast` другая высокоуровневая функция по-типу `map`. `broadcast` является обобщением `map`, так что она может делать все, что может `map` и даже больше. Синтаксис выова `broadcast` такой же как и у `map`

In [26]:
broadcast(f, [1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

и снова, мы преминяем `f` (возведение в квадрат) ко всем элементам `[1, 2, 3]` - теперь средствами "броадкастинга" `f`!

Другой способ вызова функции `broadcast` - это разместить `.` между именем функции и ее аргументами. Например,

```julia
broadcast(f, [1, 2, 3])
```
тоже самое, что и 
```julia
f.([1, 2, 3])
```

In [29]:
f.([1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Снова обратите внимание, как это отличается от вызова
```julia
f([1, 2, 3])
```
Теперь мы можем возводить в квадрат элементов вектора, а не весь  вектор целиком!

И, чтобы дойти до конца, посмотрим на разницу между

```julia
f(A)
```
и
```julia
f.(A)
```
для матрицы `A`:

In [12]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [31]:
f(A)

3×3 Matrix{Int64}:
  30   36   42
  66   81   96
 102  126  150

Как мы уже раньше видели, для матрицы `A`,
```
f(A) = A^2 = A * A
``` 

С другой стороны,

In [32]:
B = f.(A)

3×3 Matrix{Int64}:
  1   4   9
 16  25  36
 49  64  81

содержит квадраты всех элементов матрицы `A`.

Этот синтаксис с точкой для броадкастинга позволяет нам писать относительно сложные элементно-зависимые выражения в виде, похожем на  математические записи. К примеру, мы можем записать:

In [33]:
A .+ 2 .* f.(A) ./ A

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

вместо

In [34]:
broadcast(x -> x + 2 * f(x) / x, A)

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

хотя обе записи по делают то же самое.

### Упраженения

#### 6.1 
Напишите функцию `add_one` которая добавляет 1 ко входному значению.

In [7]:
function add_one(input)
    input += 1
end

add_one (generic function with 1 method)

In [3]:
@assert add_one(1) == 2

In [8]:
@assert add_one(11) == 12

#### 6.2 
Используйте `map` или `broadcast`, чтобы увеличить каждый элемент матрицы `A` на `1` и присвоить результат в переменную `A1`.

In [21]:
A1 = add_one.(A)

3×3 Matrix{Int64}:
 2  3   4
 5  6   7
 8  9  10

In [16]:
@assert A1 == [2 3 4; 5 6 7; 8 9 10]

#### 6.3 
Используйте синтаксис `broadcast` с точкой, чтобы увеличить каждый элемент матрицы `A1` на `1` и поместить его в переменную `A2`

In [22]:
A2 = add_one.(A1)

3×3 Matrix{Int64}:
 3   4   5
 6   7   8
 9  10  11

In [23]:
@assert A2 == [3 4 5; 6 7 8; 9 10 11]